<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [31]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit
import seaborn as sns
import matplotlib.pyplot as plt

# Start coding!

In [32]:
car_insurance = pd.read_csv('car_insurance.csv')

In [33]:
car_insurance.head(10)

id  age  gender  ... duis past_accidents outcome
0  569520    3       0  ...    0              0     0.0
1  750365    0       1  ...    0              0     1.0
2  199901    0       0  ...    0              0     0.0
3  478866    0       1  ...    0              0     0.0
4  731664    1       1  ...    0              1     1.0
5  877557    2       0  ...    0              3     0.0
6  930134    3       1  ...    0              3     0.0
7  461006    1       0  ...    0              0     1.0
8   68366    2       0  ...    0              0     0.0
9  445911    2       0  ...    0              0     1.0

[10 rows x 18 columns]

In [34]:
car_insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [35]:
# Finding and Filling missing Data using 'fillna'

mean_sub = {'credit_score':car_insurance['credit_score'].mean(), 'annual_mileage':car_insurance['annual_mileage'].mean()}

car_insurance.fillna(value=mean_sub, inplace= True)

In [36]:
# For more accurate mean substitution, use gruoped mean according to each income unique value

car_cleanup = car_insurance.groupby('income')['credit_score'].mean().to_dict()
print(car_cleanup)

{'middle class': 0.5228842404918247, 'poverty': 0.36513487510913784, 'upper class': 0.6048076849995434, 'working class': 0.4412387875294348}


In [37]:
car_insurance.describe(include= 'all')

id           age  ...  past_accidents       outcome
count    10000.000000  10000.000000  ...    10000.000000  10000.000000
unique            NaN           NaN  ...             NaN           NaN
top               NaN           NaN  ...             NaN           NaN
freq              NaN           NaN  ...             NaN           NaN
mean    500521.906800      1.489500  ...        1.056300      0.313300
std     290030.768758      1.025278  ...        1.652454      0.463858
min        101.000000      0.000000  ...        0.000000      0.000000
25%     249638.500000      1.000000  ...        0.000000      0.000000
50%     501777.000000      1.000000  ...        0.000000      0.000000
75%     753974.500000      2.000000  ...        2.000000      1.000000
max     999976.000000      3.000000  ...       15.000000      1.000000

[11 rows x 18 columns]

In [38]:
car_insurance.columns

Index(['id', 'age', 'gender', 'driving_experience', 'education', 'income',
       'credit_score', 'vehicle_ownership', 'vehicle_year', 'married',
       'children', 'postal_code', 'annual_mileage', 'vehicle_type',
       'speeding_violations', 'duis', 'past_accidents', 'outcome'],
      dtype='object')

In [39]:
features

Index(['age', 'gender', 'driving_experience', 'education', 'income',
       'credit_score', 'vehicle_ownership', 'vehicle_year', 'married',
       'children', 'postal_code', 'annual_mileage', 'vehicle_type',
       'speeding_violations', 'duis', 'past_accidents'],
      dtype='object')

## Identify the single feature of the data that is the best predictor of whether a customer will put in a claim (the "outcome" column), excluding the "id" column.

In [40]:
# Building and storing the models
features = car_insurance.drop(columns=['id','outcome'], inplace= False).columns
models = []

In [41]:
# Create variables for modeling and storing the results.
for col in features:
    model = logit(f"outcome ~ {col}", data= car_insurance).fit()
    models.append(model)

Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  

## Store as a DataFrame called best_feature_df, containing columns named _"best_feature"_ and _"best_accuracy"_ with the name of the feature with the highest accuracy, and the respective accuracy score.

In [42]:
features

Index(['age', 'gender', 'driving_experience', 'education', 'income',
       'credit_score', 'vehicle_ownership', 'vehicle_year', 'married',
       'children', 'postal_code', 'annual_mileage', 'vehicle_type',
       'speeding_violations', 'duis', 'past_accidents'],
      dtype='object')

In [43]:
# Creating a list to store model accuracies
accuracy = []

# Looping through the index of the models list
for feat in range(0, len(models)):
    #create a confusion matrix and storing individual metrics
    confu_matrix = models[feat].pred_table()
    #True Negative
    tn = confu_matrix[0,0]
    #False Positive
    fp = confu_matrix[0,1]
    #False Negative
    fn = confu_matrix[1,0]
    #True Positive
    tp = confu_matrix[1,1]
    # Calculating accuracy and storing the model's results
    accu_idx = (tp + tn)/(tn+fp+fn+tp)
    accuracy.append(accu_idx)
# see the accuracy
print(accuracy)


[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867, 0.6867, 0.6867, 0.6904, 0.6867, 0.6867, 0.6867, 0.6867]


In [44]:
# Finding the best performing model

best_feature = features[accuracy.index(max(accuracy))]
print(best_feature)

driving_experience


In [45]:
# Mapping the highest accuracy to the feature 

best_feature_df = pd.DataFrame({'best_feature':best_feature, 'best_accuracy':max(accuracy)}, index=[0])

best_feature_df

best_feature  best_accuracy
0  driving_experience         0.7771

In [46]:
mdl_testing = logit('outcome ~ credit_score', data= car_insurance).fit()
print(mdl_testing.params)

Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Intercept       1.950228
credit_score   -5.464491
dtype: float64
